# Testing and visualize regoinal-global cluster grid  


##  Fei Zhang
### 2018-09-24





Read in a CSV files containing the cell center coordinates of a Grid Model

Visualize the Grid centers to see if the Grid definition and spatial point mapping is correct

In [ ]:
!which python

In [ ]:
!python -V

In [ ]:
import pandas as pd
import geopandas as geopd

import matplotlib.pyplot as plt

# from IPython.display import display
# %matplotlib inline

In [ ]:
import sys
# get the pst packages into PYTHONPATH 
sys.path.append("/Softlab/Githubz/passive-seismic")  

from seismic.traveltime.cluster_grid import Grid2 
from seismic.traveltime.sort_rays import sort,sort2

In [ ]:
def csv2pdf(csvfile):
    """
    Read in a csv file into a pandas dataframe. 
    Make sure the column names match the csv files. 
    delimiter/separator is whitespace or comma
    """
    
    # read infile, tweek below
    in_pdf =  pd.read_csv(csvfile, header='infer')   #  assume there is header line=0; separator = comma,
    # finalpdf =  pd.read_csv(csvfile, header=None,  names=col_names ) #  no header line, separator = comma,
    #finalpdf =  pd.read_csv(csvfile,  sep='\s+', header=None,  names=col_names) # white space
  
    # columns you are interested?
#     col_names=['source_block', 'station_block', 'residual', 'event_number', 
#             'source_longitude','source_latitude','source_depth',  
#             'station_longitude','station_latitude', 'observed_tt', 'locations2degrees', 'station_code','SNR', 'P_or_S']   
    
    # finalpdf = in_pdf[col_names]  # take a subset of columns
    
    return in_pdf  #pandas_data_frame

In [ ]:
# my_grid= Grid2()
# mypdf = my_grid.generate_latlong_grid()

# OR read in already created CSV file:

mycsvfile='/g/data/ha3/fxz547/travel_time_tomography/Cluster_Grids/cluster_grid3D_v4.csv'

# mycsvfile = '/Softlab/travel_time_tomography/CSV_Data/cluster_grid_3d.csv'

mypdf= csv2pdf(mycsvfile)

In [ ]:

pdf2=mypdf

pdf2.columns


In [ ]:
pdf2.shape

In [ ]:
pdf2.blockn.nunique()  # number of unique values

In [ ]:
pdf2.zc.nunique()

In [ ]:
depth_list = pdf2.zc.unique()

In [ ]:
depth_list.sort()

In [ ]:
print(depth_list)

In [ ]:
pdf2.groupby(['blockn']).count().shape

In [ ]:
pdf2.groupby(['blockn']).count().head()

In [ ]:
final_pdf = pdf2.drop_duplicates(subset=['blockn'], keep='first', inplace=False)

In [ ]:
final_pdf.head()

In [ ]:
final_pdf.shape

In [ ]:
global_grid = final_pdf[final_pdf.zc > depth_list[-1] -1]

In [ ]:
global_grid.shape

In [ ]:
regional_grid = final_pdf[final_pdf.zc < depth_list[0] +1 ]

In [ ]:
regional_grid.head()

In [ ]:
regional_grid.shape

In [ ]:
# stop break

In [ ]:
final_pdf.xc.max()

In [ ]:
final_pdf.xc.min()

In [ ]:
final_pdf.yc.max()

In [ ]:
final_pdf.yc.min()

In [ ]:
# stop

# Begin to Visualize the data

# 1. Python Pandas package plot function (with basemap) 

In [ ]:
#plt.figure(); 
regional_grid.plot.scatter(x='xc', y='yc',  figsize=(12,8))

In [ ]:
global_grid.plot.scatter(x='xc', y='yc', figsize=(12,8))

# 3. Geopandas and Maps

In [ ]:
import geopandas as gpd

from shapely.geometry import mapping
from shapely.geometry import Point
from shapely.geometry import Point, Polygon, LineString, LinearRing

In [ ]:
event_locations = [Point(xy) for xy in zip(regional_grid.xc, regional_grid.yc)]
#OR pdf['geometry'] = pdf.apply(lambda z: Point(z.lon, z.lat), axis=1)
#if you want to df = df.drop(['Lon', 'Lat'], axis=1)
mycrs = {'init': 'epsg:4326'}  # WGS84
mycrs = {'init': 'epsg:4283'}  # GDA94
reg_geopdf = gpd.GeoDataFrame(regional_grid, crs=mycrs, geometry=event_locations)

In [ ]:
# myax = geopdf.plot(figsize=[20,10])


# myax.set_xlabel('Longitude')
# myax.set_ylabel('Latitude+
# title_str= "event locations"
# myax.set_title(title_str)

In [ ]:
# geopandas included shape datasets
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
myax=world.plot(alpha=0.5,  figsize=(20, 10))

myax.set_xlim([90,200])
myax.set_ylim([-70,10])
              
reg_geopdf.plot(ax=myax, marker='o', color='red', markersize=2);


In [ ]:
# geopandas included shape datasets
grid_locations = [Point(xy) for xy in zip(global_grid.xc, global_grid.yc)]
#OR pdf['geometry'] = pdf.apply(lambda z: Point(z.lon, z.lat), axis=1)
#if you want to df = df.drop(['Lon', 'Lat'], axis=1)
mycrs = {'init': 'epsg:4326'}  # WGS84
mycrs = {'init': 'epsg:4283'}  # GDA94
glob_geopdf = gpd.GeoDataFrame(global_grid, crs=mycrs, geometry=grid_locations)


In [ ]:
# geopandas included shape datasets
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
myax=world.plot(alpha=0.5,  figsize=(20, 10))

myax.set_xlim([90,200])
myax.set_ylim([-70,10])

# myax.set_xlim([160,200])
# myax.set_ylim([-10,10])


myax.set_xlabel('Longitude')
myax.set_ylabel('Latitude')
title_str= "Reginal Grid Centers (red) and Global Grid Centres (Blue) "
myax.set_title(title_str)

glob_geopdf.plot(ax=myax, marker='*', color='blue', markersize=32);

reg_geopdf.plot(ax=myax, marker='o', color='red', markersize=2);

In [ ]:
# geopandas included shape datasets
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
myax=world.plot(alpha=0.5,  figsize=(20, 10))

# myax.set_xlim([90,200])
# myax.set_ylim([-70,10])


myax.set_xlim([160,200])
myax.set_ylim([-10,10])


myax.set_xlabel('Longitude')
myax.set_ylabel('Latitude')
title_str= "Reginal Grid Centers (red) and Global Grid Centres (Blue) "
myax.set_title(title_str)

glob_geopdf.plot(ax=myax, marker='*', color='blue', markersize=32);

reg_geopdf.plot(ax=myax, marker='o', color='red', markersize=8);

In [ ]:
break

In [ ]:
for adepth in depth_list:
    hview_grid = final_pdf[final_pdf.zc == adepth]
    
    title_str="horizontal view grid at depth = %s, %s, %s, %s"%(adepth, hview_grid.shape, hview_grid.blockn.min(), hview_grid.blockn.max())
    print (title_str)
    hview_grid.plot.scatter(x='xc', y='yc',  figsize=(12,8))

# Grid properties

In [ ]:
mygrid=Grid2()

In [ ]:
mygrid.show_properties()

In [ ]:
# border corner point mapping to the grid, 
# floating point errors, discontinuity
# bugs in code 

# inside and at border of the ANZ region
mygrid.find_block_number(-0.5,189.8, 2000.0)

In [ ]:
# outside the ANZ region

mygrid.find_block_number(10, 100, 2000.0)

#  More

In [ ]:
796754/53

In [ ]:
587980/20



In [ ]:
587980/29399

In [ ]:
import numpy as np
np.linspace(0, 360, num=360*5+1)

In [ ]:
np.linspace(-90,90, num=180*5+1)

In [ ]:
import matplotlib.pyplot as plt
N = 8
y=np.zeros(N)
x1 = np.linspace(0, 10, N, endpoint=True)
x2 = np.linspace(0, 10, N, endpoint=False)
plt.plot(x1, y, 'o')

plt.plot(x2, y + 0.2, 'o')

plt.ylim([-0.4, 0.6])
# (-0.5, 1)
plt.show()

In [ ]:
print (x1, x2)

In [ ]:
x1 = np.linspace(0, 10, num=N+1)  # good grid
x2 = np.linspace(0, 10, N+1, endpoint=False)  # no good

In [ ]:
print (x1, x2)